# Job Posting Data Analysis
In this notebook, the group will be working with the [Job Posting in Singapore](https://www.kaggle.com/datasets/techsalerator/job-posting-data-in-singapore) dataset. This dataset will be used for processing, analyzing, and visualizing data.

This project is carried out by the group **DS NERDS**, under Section **S19**, which consists of the following members:
- Colobong, Franz Andrick
- Chu, Andre Benedict M. 
- Pineda, Mark Gabriel A.
- Rocha, Angelo H. 
  
The output fulfills a part of the requirements for the course Statistical Modeling and Simulation (CSMODEL). 


# Import Libraries

**TO-DO**:
Put a brief description for each module used and how it was used in the notebook.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

## Dataset Description and Collection Process

This dataset offers a comprehensive overview of job openings across various sectors in Singapore. It provides an essential resource for businesses, job seekers, and labor market analysts, and it can also be a valuable tool for people who would like to be informed about job openings and employment trends in Singapore.

The data was collected by a global data provider called **Techsalerator**, by consolidating and categorizing job-related information from diverse sources, including company websites, job boards, and recruitment agencies. 

Now, let us load the CSV file into our workspace with **'latin1'** encoding as it contains special characters (e.g., é, ñ, ’) that caused a UnicodeDecodeError with the default **'utf-8'** encoding.

In [ ]:
job_posting_df = pd.read_csv('Job Posting.csv', encoding='latin1')
job_posting_df.head(5)

## Structure of the Data

**`TO-DO`** : Provide the following:
- What each row and column represents (if tabular data)
- Number of observations
- What attributes or features are present in each observation

** use the .info() function to visualize the answer to the questions

In [ ]:
job_posting_df.info()

## Potential Implications of the Data

**`TO-DO`** : Provide the potential implications of how the data was collected on the insights that will be generated

## Key Data Fields 

This section provides a brief description of the key attributes present in the dataset:


- **Job Posting Date**: Captures the date a job is listed. This is crucial for job seekers and HR professionals to stay updated on the latest opportunities and trends.

- **Job Title**: Specifies the position being advertised. This helps in categorizing and filtering job openings based on industry roles and career interests.

- **Company Name**: Lists the hiring company. This information assists job seekers in targeting their applications and helps businesses track competitors and market trends.

- **Job Location**: Provides the job's geographic location within Singapore. Job seekers use this to find opportunities in specific areas, while employers analyze regional talent and market conditions.

- **Job Description**: Includes details about responsibilities, required qualifications, and other relevant aspects. This is vital for candidates to determine if they meet the requirements and for recruiters to communicate expectations clearly.

**`TO-DO`** : Add the description for other important fields

In [ ]:
key_data_fields = job_posting_df[['First Seen At', 'Job Opening Title', 'Job Opening URL', 'Location', 'Description']]
key_data_fields.head()

## Data Pre-Processing

Before diving into analysis, it's essential to **clean and standardize** the dataset to ensure accurate insights. In this section, we focus on preparing the data by addressing duplicates, handling missing values, correcting data types, and other data pre-processing techniques to improve data quality and ensure consistency throughout the analysis.

These steps help improve the **quality, reliability, and interpretability** of our exploratory data analysis (EDA) results.


### Remove Unnecessary Columns
Upon inspection, we can see that the **`Ticker`** column—referring to the stock ticker symbol of the company that posted the job—contains only null values.

Since this column provides no usable information for analysis or modeling, we can safely drop it from the dataset.


In [ ]:
# Check the Ticker column
null_count = job_posting_df['Ticker'].isna().sum()
print("Unique Values:", job_posting_df['Ticker'].unique())
print(f"Number of null values: {null_count}")

# Drop the column
job_posting_df = job_posting_df.drop(columns=['Ticker'])


### Remove Duplicate Job Postings

Duplicate job postings can occur when the same job is scraped or recorded multiple times. If left unaddressed, these duplicates can **bias statistical insights**, such as job availability by location or contract type. Removing them ensures each job posting is only counted once.

In [ ]:
# Remove Duplicates
job_posting_df = job_posting_df.drop_duplicates()


### Remove Entries with Missing Critical Information

For the purpose of meaningful analysis, we remove records that lack crucial details such as:

- **Location** and **Location Data**: Essential for analyzing geographic trends.
- **Contract Type**: Helps determine the nature of the job, which is important for
categorizing roles.
- **Seniority**: Provides insight into job levels, which is useful for experience-based segmentation.
- **O\*NET Family**: Offers a standardized occupational classification, more reliable and structured than a free-text category.


Missing any of these fields makes the data point less useful and **hinders the development of valid hypotheses** during exploratory data analysis (EDA).


In [ ]:
# Display the number of missing values in critical fields
print(job_posting_df[['Location', 'Location Data', 'Contract Types', 'Seniority', 'O*NET Family']].isnull().sum())

# Show how many entries are missing any of the five critical fields
print(
    "Entries missing any of the critical fields:",
    job_posting_df[['Location', 'Location Data', 'Contract Types', 'Seniority', 'O*NET Family']]
    .isnull().any(axis=1).sum(),
    "\n"
)

# Drop rows with any missing value in the critical columns
job_posting_df = job_posting_df.dropna(
    subset=['Location', 'Location Data', 'Contract Types', 'Seniority', 'O*NET Family'],
    how='any'
)

# Re-check missing values after dropping
print(job_posting_df[['Location', 'Location Data', 'Contract Types', 'Seniority', 'O*NET Family']].isnull().sum())
print(
    "Entries missing any of the critical fields after cleaning:",
    job_posting_df[['Location', 'Location Data', 'Contract Types', 'Seniority', 'O*NET Family']]
    .isnull().any(axis=1).sum()
)


### Fixing Incorrect Datatypes

To ensure that each column is using the **appropriate data type**, we begin by inspecting the current data types of all fields:


In [ ]:
job_posting_df.dtypes

### Convert Date-based Columns to DateTime Format

The **date fields** in the dataset are initially represented as generic `object` types. In this step, we convert them to their appropriate `datetime` data types and ensure they are consistently formatted.

To achieve this, we use the `pd.to_datetime()` method to parse each column. This allows us to catch any inconsistencies or formatting issues.

Any values that fail to convert (e.g., due to invalid formats or corrupted entries) are automatically set to `NaT` (*Not a Time*), enabling us to easily identify and count invalid or missing entries per column.


In [ ]:
# Define the date columns to check
date_fields = ['First Seen At', 'Last Seen At', 'Job Last Processed At']
date_df = job_posting_df[date_fields].copy()

# Convert in-place and count invalid values
for col in date_fields:
    job_posting_df[col] = pd.to_datetime(job_posting_df[col], errors='coerce')
    invalid_count = job_posting_df[col].isna().sum()
    print(f"{invalid_count:,} invalid date value(s) found in '{col}'")

In [ ]:
job_posting_df.dtypes

### Standardize Text Fields

To ensure consistency and simplify categorization, we clean key text fields by:

- **Removing leading and trailing spaces**
- **Converting all text to lowercase**

This helps avoid mismatches due to inconsistent casing (e.g., `"Full-Time"` vs `"full-time"`) or trailing whitespace (`"remote "` vs `"remote"`), especially when grouping or filtering values in analysis.

In [ ]:
# Clean and standardize text-based columns
job_posting_df['O*NET Family'] = job_posting_df['O*NET Family'].str.strip().str.lower()
job_posting_df['Keywords'] = job_posting_df['Keywords'].str.strip().str.lower()
job_posting_df['Location'] = job_posting_df['Location'].str.strip().str.lower()
job_posting_df['Seniority'] = job_posting_df['Seniority'].str.strip().str.lower()
job_posting_df['Contract Types'] = job_posting_df['Contract Types'].str.strip().str.lower()

Now that all fields have been standardized to **consistent data types and formats**, we can proceed to the next step of the data pre-processing pipeline.


## Categorizing Data for Simplified Analysis

Many fields in the dataset, such as **`Seniority`**, **`Job Category`**, **`Location`**, **`Contract Types`**, and **`Skills`**, contain a wide variety of raw or inconsistent values. While these detailed values may be useful in certain cases, they can make analysis more difficult and less interpretable at a higher level.

To address this, we apply **categorization and grouping techniques** to simplify the data. By consolidating similar or related values into broader, standardized categories, we can make comparisons and aggregations more meaningful

As part of this process, we will import a custom module called `mapper.py`, which contains predefined functions for mapping and standardizing the values in each relevant column.

In [ ]:
import mapper as mp

### Categorizing Data by Seniority

To simplify analysis based on job hierarchy, we categorize the `Seniority` field into broader groups. This allows us to analyze trends more effectively across different levels of responsibility.

We begin by inspecting the unique values in the `Seniority` column and then apply a mapping to group them into three categories:
- **Non-Managerial Position**
- **Managerial Position**
- **Executive Position**

To understand the mapping dictionary for seniority, refer to the `mapper.py` file, where it is defined under the variable `seniority_mapping`.


In [ ]:
# Check all unique values
unique_values_seniority = job_posting_df['Seniority'].unique()
print(unique_values_seniority)

In [ ]:
# Map the values and count categories
seniority_categories = job_posting_df['Seniority'].map(mp.seniority_mapping)
seniority_category_counts = seniority_categories.value_counts().sort_index()

print("=" * 50)
print(seniority_category_counts)
print("=" * 50)

### Categorizing Data by Job Field

The **job field** was categorized by analyzing the contents of the `O*NET Family` column, which provides more specific insights into the required skills, education, and training for each role. Compared to the `Category` column—which offers a broader and often more general classification—the `O*NET Family` column is a more suitable choice for identifying and analyzing job fields with greater precision.

The mapping logic for grouping related job fields is defined in the `mapper.py` file under the variable `job_fields_mappings`. Refer to that file to view how each O*NET Family is classified into a broader job field.


In [ ]:
# Check all unique values
unique_job_fields = job_posting_df['O*NET Family'].unique()
print(unique_job_fields)

In [ ]:
# Map the values and get the count of the categories
job_fields_categories = job_posting_df['O*NET Family'].map(mp.job_fields_mapping)
job_fields_category_counts = job_fields_categories.value_counts().sort_index()

print("=" * 50)
print(job_fields_category_counts)
print("=" * 50)

### Categorizing Data by Contract Types

To streamline analysis of employment structures, we categorize the values in the `Contract Types` column. This involves identifying all unique contract types present in the dataset and mapping them to broader, standardized categories.

The mapping logic is defined in the `mapper.py` file under the variable `contract_type_mapping`.


In [ ]:
split_contract_types = job_posting_df['Contract Types'].str.split(',').explode()

unique_ctypes = split_contract_types.str.strip().unique()

print(unique_ctypes)

In [ ]:
from collections import Counter

# Lowercase the contract types for case-sensitive look-ups, substring matching
mapping_ctypes_lower = {k.lower(): v for k, v in mp.contract_types_mapping.items()}

def map_ctypes_in_cell(str_keywords):
    # check for null categories
    if pd.isna(str_keywords):
        return []

    # list of contract types in lowercase
    ctypes_lower = str_keywords.lower()
    mapped_categories = []

    # loop for the contract types inside the category
    for ky, ct in mapping_ctypes_lower.items():
        # if contract types is in the list, append to list the category
        if ky in ctypes_lower:
            mapped_categories.append(ct)

    return mapped_categories

all_ct = []

# access every contract types in the dataset
for ctypes_cells in job_posting_df['Contract Types']:

    # get the category in the contract types cells then add/extend to the list
    ct = map_ctypes_in_cell(ctypes_cells)
    all_ct.extend(ct)

# Example: cell has: ['full time', 'intern', 'long term] --> ['Full Time', 'Internship/Trainee', 'Long Term']  

# counter for all categories
ct_counts = Counter(all_ct) 

# transfer to a dataframe for better mapping
df_counts = pd.DataFrame(list(ct_counts.items()), 
                        columns=['Contract Types', 'Count'])

df_counts = df_counts.sort_values('Count', ascending=False)

# Print the values
df_counts = df_counts.reset_index(drop=True)
print("=" * 50)
print(df_counts.to_string(index=False))
print("=" * 50)


### Categorizing Data by Keywords

The `Keywords` column often contains multiple entries separated by commas. To ensure accurate grouping and analysis, we first clean and split these entries into individual keywords. This helps reduce redundancy caused by inconsistent formatting (e.g., extra spaces, mixed casing).

After cleaning, we identify all unique keyword values and apply standardized categorization where needed. This step supports clearer interpretation of skillsets or role-related descriptors associated with each job posting.

The logic for processing and mapping keyword values is handled in the `mapper.py` file under the relevant keyword-related functions or mappings.

In [ ]:
# Split the contents of the keywords column
split_keywords = job_posting_df['Keywords'].str.split(',').explode()

# Then find the unique values, these mitigates redundancy a lot
unique_keywords = split_keywords.str.strip().unique()

print(unique_keywords)

The `Keywords` column contained over **500+ unique values**, many of which represented similar or related concepts. To reduce redundancy and make analysis more manageable, the values were **grouped into broader categories** such as *Programming Languages*, *Frameworks & Libraries*, *Tools & Platforms*, and others.

The mapping process was initially assisted by AI to generate a comprehensive list of groupings. A group member then manually **reviewed and verified** the mappings to ensure accuracy and consistency across the dataset.


In [ ]:
from collections import Counter

# Lowercase the keys for case-sensitive look-ups, substring matching
mapping_skills_lower = {k.lower(): v for k, v in mp.keywords_skills_mapping.items()}

def map_keywords_in_cell(str_keywords):
    # check for null categories
    if pd.isna(str_keywords):
        return []

    # list of keywords in lowercase
    keywords_lower = str_keywords.lower()
    mapped_categories = []

    # loop for the key inside the category
    for ky, ct in mapping_skills_lower.items():
        # if keyword is in the list, append to list the category
        if ky in keywords_lower:
            mapped_categories.append(ct)

    return mapped_categories

all_ct = []

# access every keyword in the dataset
for ky_cells in job_posting_df['Keywords']:

    # get the category in the keywords cells then add/extend to the list
    ct = map_keywords_in_cell(ky_cells)
    all_ct.extend(ct)

# Example: cell has: ['c++', 'mysql', 'linux] --> ['Programming Language', 'Databases', 'Operating System']  

# counter for all categories
ct_counts = Counter(all_ct) 

# transfer to a dataframe for better mapping
df_counts = pd.DataFrame(list(ct_counts.items()), 
                        columns=['Category', 'Count'])

df_counts = df_counts.sort_values('Count', ascending=False)

# Print the values
df_counts = df_counts.reset_index(drop=True)
print("=" * 50)
print(df_counts.to_string(index=False))
print("=" * 50)


### Parsing and Analyzing Location Data

To extract meaningful insights from the `Location Data` column, we begin by parsing its contents—originally stored in JSON-like strings—into structured dictionaries. We begin by creating a copy of the main dataset into `locations_df` to avoid modifying the original `job_posting_df`.

In [ ]:
locations_df = job_posting_df.copy()
# Check the contents of the Location Data
locations_df['Location Data']

This section focuses on extracting structured information from the `Location Data` column, which contains location details in JSON format. Each entry is parsed into a Python dictionary using a custom `parse_location()` function. This function handles both single dictionary entries and lists of dictionaries, returning a standardized format for further processing.

After parsing, the `json_normalize()` method is used to flatten the nested data structure, transforming the location attributes (such as `city`, `region`, and `country`) into separate columns. This results in a cleaner and more analyzable format, allowing us to explore geographic distributions—such as the number of job postings per country.


In [ ]:
# Parse data into a dictionary
def parse_location(str_location):
    try:
        # Convert the json file into a python object
        data = json.loads(str_location)

        # Takes the first element: if a list, else returns the dictionary as the 
        # first element, otherwise return the dictionary
        return data[0] if isinstance(data, list) else data
    except:
        # Return an empty list
        return {}

# Parse the location data  
locations_df['Location Data'] = locations_df['Location Data'].apply(
    parse_location
)

# Normalize Location Data into new columns and rows
locations_df = pd.json_normalize(locations_df['Location Data'])
locations_df



In [ ]:
# Display the contents by categorizing the total number of entries per country
locations_df['country'].value_counts()

### Salary Data Extraction and Preparation

To better understand the **`Salary Data`** column, we begin by creating a separate copy of the original DataFrame called `salary_df`. This ensures that all salary-related transformations and cleaning steps can be performed safely without altering the original `job_posting_df`. 

In [ ]:
salary_df = job_posting_df.copy()
salary_df['Salary Data']


Upon inspection, we notice that the salary descriptions are stored as **JSON objects**—but currently in the form of **JSON strings**.

To make this data usable, we will:

1. **Parse** each string into a Python dictionary.
2. **Normalize** the dictionary so that each key becomes its own separate column in the DataFrame.

This will give us a clearer structure, allowing us to inspect and clean salary values more effectively.


In [ ]:
salary_df = job_posting_df.copy()

# Parse json object into a dictionary
salary_df['Salary Data'] = salary_df['Salary Data'].apply(
    lambda x: json.loads(x) if isinstance(x, str) else x
)

# Normalize Salary Data into new columns and remove rows with null values
salary_df = pd.json_normalize(salary_df['Salary Data'])
salary_df

By running `salary_df.info()`, we can observe that out of thousands of job postings, only **434** entries contain salary-related information. 

Since salary is a critical detail when analyzing job data, we want to ensure our next steps focus only on entries where salary is provided. To simplify our cleaning process, we will **temporarily drop rows with null values** for salary-related fields.


In [ ]:
salary_df.info() 

# Drop rows with any null values
salary_df.dropna(inplace=True)


Now that we've removed rows with null values, we can inspect the unique values present in each field. 

In particular, the **`salary_currency`** column contains two distinct values: **USD** and **EUR**.


In [ ]:
salary_df['salary_currency'].value_counts()

After checking the `salary_currency` field, we observe that most job salaries are already in **USD**. 

To ensure consistency in our analysis, we will normalize the data by converting all **EUR** salaries to **USD** using the exchange rate as of **June 17, 2025**:

- **1 EUR = 1.15 USD**

This conversion allows us to compare salaries more accurately and ensures uniformity across the dataset.


In [ ]:
# Define conversion rate from EUR to USD
conversion_rate = 1.15

# Convert EUR to USD
for index, row in salary_df.iterrows():
    if row['salary_currency'] == 'EUR':
        salary_df.loc[index, 'salary_low'] = row['salary_low'] * conversion_rate
        salary_df.loc[index, 'salary_high'] = row['salary_high'] * conversion_rate
        salary_df.loc[index, 'salary_currency'] = 'USD'

# Drop redundant salary column 
salary_df.drop(columns=['salary_low_usd', 'salary_high_usd'], inplace=True, errors='ignore')

salary_df

Now that all the salaries are represented in **USD**, we can focus on the `salary_time_unit` column, which is categorized into three values: **hour**, **month**, and **year**. These indicate how each salary is paid.

In [ ]:
salary_df['salary_time_unit'].value_counts()

We notice that most salaries are already given on an **annual basis**. To maintain consistency and enable easier comparisons, we will convert all salaries to **annual salary**.

#### Conversion Formulas:
- **Monthly to Annual**:
  - `annual_salary = monthly_salary * 12`

- **Hourly to Annual** (assuming a standard 9-to-5 schedule):
  - `hours_per_week = 40`
  - `weeks_per_year = 52`
  - `hourly_to_annual = 40 * 52 = 2080`

In [ ]:
# Conversion factors
monthly_to_annual = 12
hours_per_week = 40
weeks_per_year = 52
hourly_to_annual = hours_per_week * weeks_per_year  # 40 * 52 = 2080

for index, row in salary_df.iterrows():
    # Convert hourly salaries to annual
    if (row['salary_time_unit'] == 'hour'):
        salary_df.loc[index, 'salary_low'] = row['salary_low'] * hourly_to_annual
        salary_df.loc[index, 'salary_high'] = row['salary_high'] * hourly_to_annual
        salary_df.loc[index, 'salary_time_unit'] = 'year'
    
    # Convert monthly salaries to annual
    elif (row['salary_time_unit'] == 'month'):
        salary_df.loc[index, 'salary_low'] = row['salary_low'] * monthly_to_annual
        salary_df.loc[index, 'salary_high'] = row['salary_high'] * monthly_to_annual
        salary_df.loc[index, 'salary_time_unit'] = 'year'

    # Retain annual salaries
    else:
        salary_df.loc[index, 'salary_low'] = row['salary_low']
        salary_df.loc[index, 'salary_high'] = row['salary_high']

salary_df


Now that all salaries are in the same currency (**USD**) and time unit (**annual**), we can focus on the `salary_low` and `salary_high` fields.

These two fields represent the **lower and upper bounds** of the offered salary range. To simplify the analysis and create a single representative salary value, we will take the **mean** of these two values.

This gives us a new column, `annual_salary`, which reflects the average offered salary for the job.

In [ ]:
salary_df['annual_salary'] = (salary_df[['salary_low', 'salary_high']].mean(axis=1))
salary_df


Now that we've created the `annual_salary` column, the original fields—`salary_low`, `salary_high`, `salary_currency`, and `salary_time_unit`—are no longer needed for further analysis.

To clean up the DataFrame and simplify its structure, we will drop these columns.


In [ ]:
salary_df.drop(columns=['salary_low', 'salary_high', 'salary_currency', 'salary_time_unit'], inplace=True)
salary_df

Now that we've cleaned and normalized the salary information into a single `annual_salary` column, we can integrate it back into the original `job_posting_df`.

We will assign this as a new column called `Annual_Salary`, allowing us to analyze job postings alongside their corresponding annual salaries.

In [ ]:
# Add the annual salary to the original job_posting_df
job_posting_df['Annual_Salary'] = salary_df['annual_salary']
job_posting_df[job_posting_df['Annual_Salary'].notnull()]

**`TO-DO`** : Check for outliers in the annual salary column

In [ ]:
# Checking for Outliers


## General Research Question

Understanding the global job landscape is essential for identifying emerging opportunities, evolving skill demands, and industry-wide shifts across different regions. By analyzing job postings and employment data, we can uncover meaningful insights into how the workforce is transforming over time.

With this in mind, the group formulated the research question:

> **What are the underlying patterns and trends in the international job market?**


**`TO-DO`** : Answer the eda questions. If applicable, use data visualization tehcniques to better display the data.

### EDA Question 1 - Annual Salary and Job Field 
Job field in this case lies on their category within the `O*NET Family` categorization in the dataset. In this EDA question, the researchers aim to understand the following:
- What is the relationship between the annual salary and the job field in the dataset?
- What is the average salary for each job field?
- Which job fields show the lowest and highest salary variability?



### EDA Question 2 - Seniority and Contract Types (and Salary Relevance)
The researchers aim for this EDA question are to identify related patterns and trends within the `Seniority` and `Contract Types` variables. They will be guided by the following questions:
- What is the relationship between seniority and contract types in the dataset?
- What is the salary distribution for each combination/category of seniority and their equal contract types?
- Are certain contract types more prevalent at specific seniority levels?


### EDA Question 3 - Locations and Skills
Skills in this case lies on their category within the `Keywords` categorization in the dataset. In this EDA question, the researchers aim to understand the following:
- What is the relationship between the skills required by companies that are outsourcing to specific locations?
- What are the prevalent skill categories that exist for each location?
- Which locations have the highest demand for specific skills?